**What you will accomplish in this chapter:**  
 You will learn how to extract, analyze, and work with the step-by-step reasoning process from your DeepSeek AI agent. You'll build tools to capture reasoning steps, analyze their structure, and use this information to improve your AI applications and understand how reasoning works.

**Understanding the Power of Reasoning Extraction**

Now that your agent can reason, let's dive into the fascinating world of extracting and analyzing exactly how it thinks step by step. This is where the magic of reasoning agents really shines—you get complete visibility into the AI's thought process.

Traditional AI models are like magicians who never reveal their tricks. But DeepSeek reasoning agents are more like teachers who show every step of their work on the whiteboard. This transparency is invaluable for:

·       **Debugging**: Understanding why an AI made a specific decision

·       **Learning**: Studying effective problem-solving techniques

·       **Trust Building**: Verifying the logic behind AI recommendations

·       **Quality Assurance**: Identifying flawed reasoning before it affects results

·       **Process Improvement**: Refining how your AI approaches different types of problems

**Step 1: Create a Reasoning Extraction System**

1\.   	**Create a new file for reasoning analysis**:

o   In your Codespace Explorer panel, right-click and select "**New File**"

o   Name the file: reasoning\_extractor.py

o   This will contain all your reasoning analysis tools

2\.   	**Set up the basic reasoning extraction framework**:

o   Open reasoning\_extractor.py and add this foundation:

In [ ]:
import re  
 import json  
 from typing import List, Dict, Any, Optional  
 from dataclasses import dataclass  
 from enum import Enum

 class ReasoningStepType(Enum):  
 	"""Types of reasoning steps we can identify"""  
 	UNDERSTANDING \= "understanding"  
 	ANALYSIS \= "analysis"  
 	CALCULATION \= "calculation"  
 	EVALUATION \= "evaluation"  
 	CONCLUSION \= "conclusion"  
 	QUESTION \= "question"  
 	OTHER \= "other"

 @dataclass  
 class ReasoningStep:  
 	"""Represents a single step in the reasoning process"""  
 	step\_number: int  
 	content: str  
 	step\_type: ReasoningStepType  
 	confidence: float \= 0.8  
 	keywords: List\[str\] \= None  
 	  
 	def \_\_post\_init\_\_(self):  
     	if self.keywords is None:  
         	self.keywords \= \[\]

 class ReasoningExtractor:  
 	"""Extracts and analyzes reasoning steps from AI agent responses"""  
 	  
 	def \_\_init\_\_(self):  
     	\# Patterns to identify different types of reasoning steps  
     	self.step\_patterns \= \[  
         	r'Step \\d+:?\\s\*(.+)',       	\# "Step 1: Description"  
             r'\\d+\\.\\s+(.+)',            	\# "1. Description"  
         	r'\\d+\\)\\s+(.+)',            	\# "1) Description"  
         	r'First\[,:\]?\\s\*(.+)',       	\# "First, ..."  
         	r'Next\[,:\]?\\s\*(.+)',        	\# "Next, ..."  
         	r'Then\[,:\]?\\s\*(.+)',        	\# "Then, ..."  
         	r'Finally\[,:\]?\\s\*(.+)',     	\# "Finally, ..."  
             r'Therefore\[,:\]?\\s\*(.+)',   	\# "Therefore, ..."  
         	r'Let me\\s+(.+)',           	\# "Let me think..."  
         	r'I need to\\s+(.+)',        	\# "I need to..."  
     	\]  
     	  
     	\# Keywords that help classify reasoning types  
     	self.classification\_keywords \= {  
             ReasoningStepType.UNDERSTANDING: \[  
             	'understand', 'interpret', 'means', 'asking', 'question',  
             	'problem', 'given', 'we have', 'the task'  
         	\],  
         	ReasoningStepType.ANALYSIS: \[  
             	'analyze', 'examine', 'look at', 'consider', 'evaluate',  
             	'break down', 'identify', 'determine'  
         	\],  
             ReasoningStepType.CALCULATION: \[  
             	'calculate', 'compute', 'count', 'add', 'subtract',  
             	'multiply', 'divide', 'equals', 'result', 'total'  
         	\],  
         	ReasoningStepType.EVALUATION: \[  
             	'compare', 'assess', 'judge', 'weigh', 'decide',  
             	'choose', 'select', 'prefer', 'better', 'worse'  
         	\],  
         	ReasoningStepType.CONCLUSION: \[  
             	'conclude', 'therefore', 'thus', 'so', 'hence',  
    	         'final', 'answer', 'result', 'summary'  
         	\],  
         	ReasoningStepType.QUESTION: \[  
             	'what if', 'how about', 'why', 'what', 'when',  
             	'where', 'which', 'should I', 'could we'  
         	\]  
     	}  
     	  
     	print("✅ Reasoning Extractor initialized")

**Step 2: Implement Step Extraction Logic**

1\.   	**Add the core extraction method**:

o   Continue in reasoning\_extractor.py, adding these methods to the ReasoningExtractor class:

In [ ]:
	def extract\_reasoning\_steps(self, response\_text: str) \-\> List\[ReasoningStep\]:  
     	"""  
     	Extract individual reasoning steps from agent response text  
     	  
     	Args:  
         	response\_text: The full response from the reasoning agent  
         	  
     	Returns:  
         	List of ReasoningStep objects  
     	"""  
     	steps \= \[\]  
     	lines \= response\_text.split('\\n')  
     	current\_step\_number \= 0  
     	  
     	for line in lines:  
         	line \= line.strip()  
         	if not line:  
             	continue  
         	  
         	\# Check if this line matches any reasoning step pattern  
         	step\_content \= self.\_extract\_step\_content(line)  
         	  
         	if step\_content:  
             	current\_step\_number \+= 1  
             	  
             	\# Classify the type of reasoning step  
             	step\_type \= self.\_classify\_step\_type(step\_content)  
             	  
             	\# Extract key terms from the step  
             	keywords \= self.\_extract\_keywords(step\_content)  
             	  
             	\# Calculate confidence based on content indicators  
             	confidence \= self.\_calculate\_step\_confidence(step\_content)  
             	  
             	step \= ReasoningStep(  
                     step\_number=current\_step\_number,  
                 	content=step\_content,  
                 	step\_type=step\_type,  
                     confidence=confidence,  
                 	keywords=keywords  
             	)  
             	  
             	steps.append(step)  
     	  
     	return steps  
 	  
 	def \_extract\_step\_content(self, line: str) \-\> Optional\[str\]:  
     	"""Extract the actual content from a line that might contain a reasoning step"""  
     	for pattern in self.step\_patterns:  
         	match \= re.search(pattern, line, re.IGNORECASE)  
         	if match:  
             	return match.group(1).strip()  
     	  
     	\# If no specific pattern matches, check if line looks like reasoning  
     	if self.\_looks\_like\_reasoning(line):  
         	return line  
     	  
     	return None  
 	  
 	def \_looks\_like\_reasoning(self, line: str) \-\> bool:  
     	"""Determine if a line looks like it contains reasoning"""  
     	reasoning\_indicators \= \[  
         	'because', 'since', 'therefore', 'thus', 'so',  
         	'first', 'second', 'third', 'next', 'then', 'finally',  
         	'let me', 'i need', 'we can', 'this means', 'in other words'  
     	\]  
     	  
     	line\_lower \= line.lower()  
     	return any(indicator in line\_lower for indicator in reasoning\_indicators)  
 	  
 	def \_classify\_step\_type(self, content: str) \-\> ReasoningStepType:  
     	"""Classify what type of reasoning step this is"""  
     	content\_lower \= content.lower()  
     	  
     	\# Count keyword matches for each category  
     	best\_category \= ReasoningStepType.OTHER  
     	best\_score \= 0  
     	  
     	for category, keywords in self.classification\_keywords.items():  
         	score \= sum(1 for keyword in keywords if keyword in content\_lower)  
         	if score \> best\_score:  
             	best\_score \= score  
             	best\_category \= category  
     	  
     	return best\_category  
 	  
 	def \_extract\_keywords(self, content: str) \-\> List\[str\]:  
     	"""Extract important keywords from a reasoning step"""  
     	\# Look for mathematical terms, logical connectors, and key concepts  
     	keyword\_patterns \= \[  
             r'\\b(?:because|since|therefore|thus|however|although)\\b',  
             r'\\b(?:first|second|third|next|then|finally)\\b',  
             r'\\b(?:calculate|count|analyze|evaluate|determine)\\b',  
         	r'\\b(?:equal|equals|result|answer|solution)\\b',  
         	r'\\d+(?:\\.\\d+)?',  \# Numbers  
     	\]  
     	  
     	keywords \= \[\]  
     	for pattern in keyword\_patterns:  
         	matches \= re.findall(pattern, content, re.IGNORECASE)  
             keywords.extend(\[match.lower() for match in matches\])  
     	  
     	return list(set(keywords))  \# Remove duplicates  
 	  
 	def \_calculate\_step\_confidence(self, content: str) \-\> float:  
     	"""Calculate confidence score for a reasoning step"""  
     	confidence \= 0.5  \# Base confidence  
     	  
     	\# Higher confidence for longer, more detailed steps  
     	if len(content) \> 50:  
         	confidence \+= 0.2  
     	  
     	\# Higher confidence for steps with logical connectors  
     	logical\_connectors \= \['because', 'therefore', 'since', 'thus', 'so'\]  
     	if any(connector in content.lower() for connector in logical\_connectors):  
         	confidence \+= 0.2  
     	  
     	\# Higher confidence for steps with specific details  
     	if re.search(r'\\d+', content):  \# Contains numbers  
         	confidence \+= 0.1  
     	  
     	return min(confidence, 1.0)


**Step 3: Add Analysis and Quality Assessment**

1\.   	**Add methods to analyze reasoning quality**:

o   Continue adding these methods to the ReasoningExtractor class:


In [ ]:
	def analyze\_reasoning\_quality(self, steps: List\[ReasoningStep\]) \-\> Dict\[str, Any\]:  
     	"""  
     	Analyze the overall quality of the reasoning process  
     	  
     	Args:  
         	steps: List of reasoning steps to analyze  
         	  
     	Returns:  
         	Dictionary with quality metrics and analysis  
     	"""  
     	if not steps:  
         	return {  
             	'quality\_score': 0.0,  
             	'total\_steps': 0,  
             	'step\_distribution': {},  
             	'strengths': \[\],  
             	'weaknesses': \['No reasoning steps found'\],  
             	'recommendations': \['Ensure the agent provides step-by-step reasoning'\]  
         	}  
     	  
     	analysis \= {  
         	'total\_steps': len(steps),  
         	'average\_confidence': sum(step.confidence for step in steps) / len(steps),  
         	'step\_distribution': {},  
         	'strengths': \[\],  
         	'weaknesses': \[\],  
         	'recommendations': \[\]  
     	}  
     	  
     	\# Analyze step type distribution  
     	for step in steps:  
         	step\_type \= step.step\_type.value  
             analysis\['step\_distribution'\]\[step\_type\] \= analysis\['step\_distribution'\].get(step\_type, 0\) \+ 1  
     	  
     	\# Assess reasoning completeness  
     	step\_types\_present \= set(step.step\_type for step in steps)  
     	  
     	if ReasoningStepType.UNDERSTANDING in step\_types\_present:  
             analysis\['strengths'\].append('Shows problem understanding')  
     	else:  
             analysis\['weaknesses'\].append('Missing problem understanding step')  
     	  
     	if ReasoningStepType.ANALYSIS in step\_types\_present:  
             analysis\['strengths'\].append('Includes analytical thinking')  
     	else:  
             analysis\['weaknesses'\].append('Limited analytical reasoning')  
     	  
     	if ReasoningStepType.CONCLUSION in step\_types\_present:  
             analysis\['strengths'\].append('Provides clear conclusions')  
     	else:  
             analysis\['weaknesses'\].append('No clear conclusion or summary')  
     	  
   	  \# Calculate quality score  
     	completeness\_score \= len(step\_types\_present) / len(ReasoningStepType)  
     	length\_score \= min(len(steps) / 5, 1.0)  \# Optimal around 5 steps  
     	confidence\_score \= analysis\['average\_confidence'\]  
     	  
     	analysis\['quality\_score'\] \= (completeness\_score \+ length\_score \+ confidence\_score) / 3  
     	  
     	\# Generate recommendations  
     	if analysis\['quality\_score'\] \< 0.6:  
             analysis\['recommendations'\].append('Consider providing more detailed reasoning steps')  
     	  
     	if analysis\['average\_confidence'\] \< 0.7:  
             analysis\['recommendations'\].append('Reasoning steps could be more specific and detailed')  
     	  
     	if len(steps) \< 3:  
             analysis\['recommendations'\].append('Break down the problem into more steps for clarity')  
     	  
     	return analysis  
 	  
 	def get\_reasoning\_summary(self, response\_text: str) \-\> Dict\[str, Any\]:  
     	"""  
     	Get a complete summary of reasoning from agent response  
     	  
     	Args:  
         	response\_text: Full agent response text  
         	  
     	Returns:  
         	Comprehensive reasoning analysis  
     	"""  
     	steps \= self.extract\_reasoning\_steps(response\_text)  
     	quality\_analysis \= self.analyze\_reasoning\_quality(steps)  
     	  
     	summary \= {  
         	'original\_response\_length': len(response\_text),  
         	'reasoning\_steps': \[  
             	{  
                 	'step\_number': step.step\_number,  
                 	'content': step.content,  
                 	'type': step.step\_type.value,  
                 	'confidence': step.confidence,  
                 	'keywords': step.keywords  
             	}  
             	for step in steps  
         	\],  
         	'quality\_metrics': quality\_analysis,  
         	'reasoning\_flow': self.\_analyze\_reasoning\_flow(steps)  
     	}  
     	  
     	return summary  
 	  
 	def \_analyze\_reasoning\_flow(self, steps: List\[ReasoningStep\]) \-\> Dict\[str, Any\]:  
     	"""Analyze the logical flow of reasoning steps"""  
     	if len(steps) \< 2:  
         	return {'flow\_quality': 'insufficient', 'coherence': 0.0}  
     	  
     	flow\_analysis \= {  
         	'flow\_quality': 'good',  
         	'coherence': 0.8,  
         	'transitions': \[\]  
     	}  
     	  
     	\# Analyze transitions between steps  
     	for i in range(len(steps) \- 1):  
         	current\_step \= steps\[i\]  
         	next\_step \= steps\[i \+ 1\]  
         	  
         	transition \= {  
             	'from\_type': current\_step.step\_type.value,  
             	'to\_type': next\_step.step\_type.value,  
             	'logical': self.\_is\_logical\_transition(current\_step.step\_type, next\_step.step\_type)  
         	}  
         	  
             flow\_analysis\['transitions'\].append(transition)  
     	  
     	\# Calculate overall coherence  
     	logical\_transitions \= sum(1 for t in flow\_analysis\['transitions'\] if t\['logical'\])  
     	flow\_analysis\['coherence'\] \= logical\_transitions / len(flow\_analysis\['transitions'\])  
     	  
     	return flow\_analysis  
 	  
 	def \_is\_logical\_transition(self, from\_type: ReasoningStepType, to\_type: ReasoningStepType) \-\> bool:  
     	"""Determine if a transition between step types is logical"""  
     	\# Define logical transition patterns  
     	logical\_patterns \= {  
             ReasoningStepType.UNDERSTANDING: \[ReasoningStepType.ANALYSIS, ReasoningStepType.CALCULATION\],  
         	ReasoningStepType.ANALYSIS: \[ReasoningStepType.CALCULATION, ReasoningStepType.EVALUATION\],  
             ReasoningStepType.CALCULATION: \[ReasoningStepType.EVALUATION, ReasoningStepType.CONCLUSION\],  
         	ReasoningStepType.EVALUATION: \[ReasoningStepType.CONCLUSION\],  
         	ReasoningStepType.QUESTION: \[ReasoningStepType.ANALYSIS, ReasoningStepType.UNDERSTANDING\]  
     	}  
     	  
     	return to\_type in logical\_patterns.get(from\_type, \[ReasoningStepType.OTHER\])


**Step 4: Test Reasoning Extraction with Real Examples**

1\.   	**Create a test file to demonstrate reasoning extraction**:

o   Create a new file called test\_reasoning\_extraction.py:


In [ ]:
from deepseek\_agent import reasoning\_agent\_enhanced  
 from reasoning\_extractor import ReasoningExtractor

 def test\_reasoning\_extraction():  
 	"""Test reasoning extraction with various types of problems"""  
 	print("🧠 Testing Reasoning Step Extraction...")  
 	print("="\*60)  
 	  
 	\# Initialize the reasoning extractor  
 	extractor \= ReasoningExtractor()  
 	  
 	\# Test cases with different reasoning types  
 	test\_cases \= \[  
     	{  
         	'name': 'Letter Counting Problem',  
         	'question': "How many R's are in the word 'strawberry'? Show your reasoning step by step."  
     	},  
     	{  
         	'name': 'Mathematical Problem',  
         	'question': "What is 15% of 240? Please show your calculation steps."  
     	},  
     	{  
         	'name': 'Logic Problem',  
         	'question': "If all cats are mammals, and Fluffy is a cat, what can we conclude about Fluffy? Show your logical reasoning."  
     	}  
 	\]  
 	  
 	for i, test\_case in enumerate(test\_cases, 1):  
     	print(f"\\n{'='\*20} TEST {i}: {test\_case\['name'\]} {'='\*20}")  
     	print(f"Question: {test\_case\['question'\]}")  
     	print("-"\*60)  
     	  
     	try:  
         	\# Get response from agent  
         	result \= reasoning\_agent\_enhanced.ask\_question(  
             	test\_case\['question'\],  
             	show\_reasoning=False  \# We'll extract it ourselves  
         	)  
         	  
         	if result\['status'\] \== 'success':  
             	\# Extract reasoning steps  
             	response\_text \= result\['response'\]  
             	summary \= extractor.get\_reasoning\_summary(response\_text)  
             	  
             	print(f"📊 EXTRACTION RESULTS:")  
             	print(f"   Steps found: {len(summary\['reasoning\_steps'\])}")  
             	print(f"   Quality score: {summary\['quality\_metrics'\]\['quality\_score'\]:.2f}")  
             	print(f"   Flow coherence: {summary\['reasoning\_flow'\]\['coherence'\]:.2f}")  
             	  
             	print(f"\\n📋 EXTRACTED REASONING STEPS:")  
             	for step in summary\['reasoning\_steps'\]:  
                 	print(f"   Step {step\['step\_number'\]}: {step\['content'\]}")  
                 	print(f"   │ Type: {step\['type'\]}")  
                 	print(f"   │ Confidence: {step\['confidence'\]:.2f}")  
                 	print(f"   │ Keywords: {', '.join(step\['keywords'\]) if step\['keywords'\] else 'None'}")  
                 	print(f"   └─────")  
             	  
             	\# Show quality analysis  
            	 quality \= summary\['quality\_metrics'\]  
             	print(f"\\n💡 QUALITY ANALYSIS:")  
             	print(f"   Strengths: {', '.join(quality\['strengths'\]) if quality\['strengths'\] else 'None identified'}")  
             	if quality\['weaknesses'\]:  
                 	print(f"   Areas for improvement: {', '.join(quality\['weaknesses'\])}")  
             	if quality\['recommendations'\]:  
                 	print(f"   Recommendations: {', '.join(quality\['recommendations'\])}")  
         	  
         	else:  
             	print(f"❌ Agent request failed: {result\['response'\]}")  
             	  
     	except Exception as e:  
         	print(f"❌ Test failed with error: {str(e)}")  
 	  
 	print(f"\\n{'='\*60}")  
 	print("🎉 Reasoning extraction testing completed\!")

 if \_\_name\_\_ \== "\_\_main\_\_":  
 	test\_reasoning\_extraction()

2\.   	**Run the reasoning extraction test**:

o   Save the file and in your terminal run: python test\_reasoning\_extraction.py

o   Watch as the system extracts and analyzes reasoning steps from real agent responses

**Step 5: Create Advanced Reasoning Analysis Tools**

1\.   	**Add interactive reasoning exploration**:

o   Create interactive\_reasoning\_analyzer.py:

In [ ]:
from deepseek\_agent import reasoning\_agent\_enhanced  
 from reasoning\_extractor import ReasoningExtractor  
 import json

 def interactive\_reasoning\_analyzer():  
 	"""Interactive tool for analyzing agent reasoning"""  
 	print("🔬 Interactive Reasoning Analyzer")  
 	print("="\*50)  
 	print("Ask questions and get detailed reasoning analysis\!")  
 	print("Type 'quit' to exit, 'help' for instructions")  
 	print("-"\*50)  
 	  
 	extractor \= ReasoningExtractor()  
 	  
 	while True:  
     	try:  
         	user\_input \= input("\\n🤔 Your question: ").strip()  
         	  
         	if user\_input.lower() in \['quit', 'exit', 'q'\]:  
             	print("👋 Goodbye\! Happy reasoning\!")  
             	break  
         	  
         	elif user\_input.lower() \== 'help':  
             	print("\\n💡 This tool shows you how the AI thinks:")  
             	print("- Ask any question that requires reasoning")  
             	print("- The system will extract and analyze reasoning steps")  
             	print("- You'll see step types, confidence scores, and quality metrics")  
             	print("- Good examples: math problems, logic puzzles, analysis tasks")  
             	continue  
         	  
         	elif not user\_input:  
             	print("Please enter a question or 'help' for instructions.")  
             	continue  
         	  
         	\# Get agent response  
         	print("🔄 Getting agent response...")  
         	result \= reasoning\_agent\_enhanced.ask\_question(user\_input, show\_reasoning=False)  
         	  
         	if result\['status'\] \== 'success':  
             	\# Extract and analyze reasoning  
             	summary \= extractor.get\_reasoning\_summary(result\['response'\])  
             	  
             	print(f"\\n📖 ORIGINAL RESPONSE:")  
             	print("-"\*30)  
             	print(result\['response'\])  
             	  
             	print(f"\\n🧠 REASONING ANALYSIS:")  
             	print("-"\*30)  
             	  
             	if summary\['reasoning\_steps'\]:  
                 	for step in summary\['reasoning\_steps'\]:  
                     	print(f"Step {step\['step\_number'\]}: {step\['content'\]}")  
                         print(f"   Type: {step\['type'\].title()} | Confidence: {step\['confidence'\]:.2f}")  
                     	if step\['keywords'\]:  
                             print(f"   Keywords: {', '.join(step\['keywords'\])}")  
                     	print()  
                 	  
                 	\# Quality summary  
                 	quality \= summary\['quality\_metrics'\]  
                 	print(f"📊 QUALITY METRICS:")  
                 	print(f"   Overall Score: {quality\['quality\_score'\]:.2f}/1.0")  
                 	print(f"   Total Steps: {quality\['total\_steps'\]}")  
                 	print(f"   Average Confidence: {quality\['average\_confidence'\]:.2f}")  
                 	print(f"   Flow Coherence: {summary\['reasoning\_flow'\]\['coherence'\]:.2f}")  
                 	  
                 	if quality\['strengths'\]:  
                         print(f"   ✅ Strengths: {', '.join(quality\['strengths'\])}")  
                 	if quality\['weaknesses'\]:  
                         print(f"   ⚠️  Areas to improve: {', '.join(quality\['weaknesses'\])}")  
             	  
             	else:  
                 	print("No clear reasoning steps detected in the response.")  
                 	print("Try asking a question that requires step-by-step thinking.")  
         	  
         	else:  
             	print(f"❌ Agent error: {result\['response'\]}")  
     	  
     	except KeyboardInterrupt:  
         	print("\\n\\n👋 Interrupted. Goodbye\!")  
         	break  
     	except Exception as e:  
         	print(f"❌ Error: {str(e)}")

 if \_\_name\_\_ \== "\_\_main\_\_":  
 	interactive\_reasoning\_analyzer()

2\.   	**Create a reasoning comparison tool**:

o   Create compare\_reasoning\_approaches.py:


In [ ]:
from deepseek\_agent import reasoning\_agent\_enhanced  
 from reasoning\_extractor import ReasoningExtractor

 def compare\_reasoning\_approaches():  
 	"""Compare reasoning approaches for the same problem"""  
 	print("⚖️ Reasoning Approach Comparison Tool")  
 	print("="\*50)  
 	  
 	extractor \= ReasoningExtractor()  
 	  
 	\# Test the same problem with different prompting approaches  
 	base\_problem \= "A store sells apples for $2 per pound and oranges for $3 per pound. If someone buys 4 pounds of apples and 2 pounds of oranges, how much do they spend in total?"  
 	  
 	approaches \= \[  
     	{  
         	'name': 'Direct Question',  
         	'prompt': base\_problem  
     	},  
     	{  
         	'name': 'Step-by-Step Request',  
         	'prompt': f"{base\_problem} Please show your work step by step."  
     	},  
     	{  
         	'name': 'Detailed Analysis Request',  
         	'prompt': f"{base\_problem} Please break this down into clear steps: 1\) Identify what we know, 2\) Calculate each part, 3\) Add them together, 4\) State your final answer."  
     	}  
 	\]  
 	  
 	results \= \[\]  
 	  
 	for i, approach in enumerate(approaches, 1):  
     	print(f"\\n{'='\*20} APPROACH {i}: {approach\['name'\]} {'='\*20}")  
     	print(f"Prompt: {approach\['prompt'\]}")  
     	print("-"\*60)  
     	  
     	try:  
         	\# Get agent response  
         	result \= reasoning\_agent\_enhanced.ask\_question(approach\['prompt'\], show\_reasoning=False)  
         	  
         	if result\['status'\] \== 'success':  
             	\# Analyze reasoning  
             	summary \= extractor.get\_reasoning\_summary(result\['response'\])  
             	  
             	approach\_result \= {  
                 	'name': approach\['name'\],  
                 	'response': result\['response'\],  
                 	'steps\_found': len(summary\['reasoning\_steps'\]),  
                 	'quality\_score': summary\['quality\_metrics'\]\['quality\_score'\],  
                 	'coherence': summary\['reasoning\_flow'\]\['coherence'\],  
                 	'step\_types': list(set(step\['type'\] for step in summary\['reasoning\_steps'\])),  
                 	'strengths': summary\['quality\_metrics'\]\['strengths'\],  
                 	'weaknesses': summary\['quality\_metrics'\]\['weaknesses'\]  
             	}  
             	  
                 results.append(approach\_result)  
             	  
             	print(f"📊 Quick Analysis:")  
             	print(f"   Steps extracted: {approach\_result\['steps\_found'\]}")  
             	print(f"   Quality score: {approach\_result\['quality\_score'\]:.2f}")  
             	print(f"   Coherence: {approach\_result\['coherence'\]:.2f}")  
             	print(f"   Step types: {', '.join(approach\_result\['step\_types'\])}")  
             	  
         	else:  
             	print(f"❌ Failed: {result\['response'\]}")  
     	  
     	except Exception as e:  
         	print(f"❌ Error: {str(e)}")  
 	  
 	\# Compare results  
 	print(f"\\n{'='\*60}")  
 	print("📊 COMPARISON SUMMARY:")  
 	print("="\*60)  
 	  
 	if results:  
     	\# Find best approach  
     	best\_quality \= max(results, key=lambda x: x\['quality\_score'\])  
     	most\_detailed \= max(results, key=lambda x: x\['steps\_found'\])  
     	most\_coherent \= max(results, key=lambda x: x\['coherence'\])  
     	  
     	print(f"🏆 Best Overall Quality: {best\_quality\['name'\]} (Score: {best\_quality\['quality\_score'\]:.2f})")  
     	print(f"📋 Most Detailed: {most\_detailed\['name'\]} ({most\_detailed\['steps\_found'\]} steps)")  
     	print(f"🔗 Most Coherent: {most\_coherent\['name'\]} (Coherence: {most\_coherent\['coherence'\]:.2f})")  
     	  
     	print(f"\\n💡 INSIGHTS:")  
     	print("- Different prompting approaches can significantly affect reasoning quality")  
     	print("- More specific requests often lead to better step-by-step reasoning")  
     	print("- The best approach depends on your specific use case and quality requirements")  
 	  
 	print(f"\\n🎉 Reasoning comparison completed\!")

 if \_\_name\_\_ \== "\_\_main\_\_":  
 	compare\_reasoning\_approaches()


**Step 6: Practice with Different Problem Types**

1\.   	**Run the interactive analyzer**:

o   Save your files and run: python interactive\_reasoning\_analyzer.py

o   Try different types of questions to see how reasoning extraction works:

§  **Mathematical problems**: "What is the square root of 144 and how did you find it?"

§  **Logic puzzles**: "If all birds can fly, and penguins are birds, but penguins cannot fly, what's wrong with this reasoning?"

§  **Code analysis**: "What might be wrong with this code: for i in range(10): print(i/0)?"

2\.   	**Run the comparison tool**:

o   Execute: python compare\_reasoning\_approaches.py

o   Observe how different prompting strategies affect the quality and detail of reasoning

**Step 7: Build a Reasoning Quality Dashboard**

1\.   	**Create a comprehensive analysis dashboard**:

o   Create reasoning\_dashboard.py:


In [ ]:
from deepseek\_agent import reasoning\_agent\_enhanced  
 from reasoning\_extractor import ReasoningExtractor  
 import json  
 from datetime import datetime

 def create\_reasoning\_report(question: str, save\_to\_file: bool \= True):  
 	"""Create a comprehensive reasoning quality report"""  
 	print("📊 Creating Reasoning Quality Report...")  
 	  
 	extractor \= ReasoningExtractor()  
 	  
 	\# Get agent response  
 	result \= reasoning\_agent\_enhanced.ask\_question(question, show\_reasoning=False)  
 	  
 	if result\['status'\] \!= 'success':  
     	print(f"❌ Could not get agent response: {result\['response'\]}")  
     	return  
 	  
 	\# Analyze reasoning  
 	summary \= extractor.get\_reasoning\_summary(result\['response'\])  
 	  
 	\# Create comprehensive report  
 	report \= {  
     	'timestamp': datetime.now().isoformat(),  
     	'question': question,  
     	'agent\_response': result\['response'\],  
     	'reasoning\_analysis': summary,  
     	'recommendations': \[\]  
 	}  
 	  
 	\# Generate specific recommendations  
 	quality \= summary\['quality\_metrics'\]  
 	if quality\['quality\_score'\] \< 0.7:  
         report\['recommendations'\].append("Consider rephrasing the question to encourage more detailed reasoning")  
 	  
 	if len(summary\['reasoning\_steps'\]) \< 3:  
         report\['recommendations'\].append("Ask for step-by-step breakdown to get more detailed reasoning")  
 	  
 	if summary\['reasoning\_flow'\]\['coherence'\] \< 0.7:  
         report\['recommendations'\].append("The reasoning flow could be more logical \- consider asking for clearer connections between steps")  
 	  
 	\# Display report  
 	print(f"\\n📋 REASONING QUALITY REPORT")  
 	print("="\*60)  
 	print(f"Question: {question}")  
 	print(f"Generated: {report\['timestamp'\]}")  
 	print(f"\\n📊 SUMMARY METRICS:")  
 	print(f"   Quality Score: {quality\['quality\_score'\]:.2f}/1.0")  
 	print(f"   Reasoning Steps: {len(summary\['reasoning\_steps'\])}")  
 	print(f"   Flow Coherence: {summary\['reasoning\_flow'\]\['coherence'\]:.2f}")  
 	print(f"   Average Confidence: {quality\['average\_confidence'\]:.2f}")  
 	  
 	print(f"\\n🎯 DETAILED BREAKDOWN:")  
 	for step in summary\['reasoning\_steps'\]:  
     	print(f"   Step {step\['step\_number'\]}: {step\['type'\]} (Confidence: {step\['confidence'\]:.2f})")  
     	print(f"  	{step\['content'\]\[:80\]}{'...' if len(step\['content'\]) \> 80 else ''}")  
 	  
 	if quality\['strengths'\]:  
     	print(f"\\n✅ STRENGTHS:")  
     	for strength in quality\['strengths'\]:  
         	print(f"   • {strength}")  
 	  
 	if quality\['weaknesses'\]:  
     	print(f"\\n⚠️  AREAS FOR IMPROVEMENT:")  
     	for weakness in quality\['weaknesses'\]:  
         	print(f"   • {weakness}")  
 	  
 	if report\['recommendations'\]:  
     	print(f"\\n💡 RECOMMENDATIONS:")  
     	for rec in report\['recommendations'\]:  
         	print(f"   • {rec}")  
 	  
 	\# Save to file if requested  
 	if save\_to\_file:  
     	filename \= f"reasoning\_report\_{datetime.now().strftime('%Y%m%d\_%H%M%S')}.json"  
     	with open(filename, 'w') as f:  
         	json.dump(report, f, indent=2)  
     	print(f"\\n💾 Report saved to: {filename}")  
 	  
 	return report

 def batch\_reasoning\_analysis(questions: List\[str\]):  
 	"""Analyze reasoning quality for multiple questions"""  
 	print("📈 Batch Reasoning Quality Analysis")  
 	print("="\*50)  
 	  
 	results \= \[\]  
 	extractor \= ReasoningExtractor()  
 	  
 	for i, question in enumerate(questions, 1):  
         print(f"\\n\[{i}/{len(questions)}\] Analyzing: {question\[:50\]}...")  
     	  
     	try:  
         	result \= reasoning\_agent\_enhanced.ask\_question(question, show\_reasoning=False)  
         	if result\['status'\] \== 'success':  
             	summary \= extractor.get\_reasoning\_summary(result\['response'\])  
             	  
             	results.append({  
                 	'question': question,  
                 	'quality\_score': summary\['quality\_metrics'\]\['quality\_score'\],  
                 	'steps\_count': len(summary\['reasoning\_steps'\]),  
                 	'coherence': summary\['reasoning\_flow'\]\['coherence'\]  
             	})  
             	  
             	print(f"   Quality: {summary\['quality\_metrics'\]\['quality\_score'\]:.2f} | Steps: {len(summary\['reasoning\_steps'\])}")  
         	else:  
             	print(f"   ❌ Failed: {result\['response'\]}")  
     	  
     	except Exception as e:  
         	print(f"   ❌ Error: {str(e)}")  
 	  
 	\# Summary statistics  
 	if results:  
     	avg\_quality \= sum(r\['quality\_score'\] for r in results) / len(results)  
     	avg\_steps \= sum(r\['steps\_count'\] for r in results) / len(results)  
     	avg\_coherence \= sum(r\['coherence'\] for r in results) / len(results)  
     	  
     	print(f"\\n📊 BATCH ANALYSIS SUMMARY:")  
     	print(f"   Questions analyzed: {len(results)}")  
     	print(f"   Average quality score: {avg\_quality:.2f}")  
     	print(f"   Average reasoning steps: {avg\_steps:.1f}")  
     	print(f"   Average coherence: {avg\_coherence:.2f}")  
 	  
 	return results

 if \_\_name\_\_ \== "\_\_main\_\_":  
 	\# Example usage  
 	sample\_question \= "How would you design a simple password strength checker? Walk me through your thinking process."  
     create\_reasoning\_report(sample\_question)

2\.   	**Test the dashboard**:

o   Run: python reasoning\_dashboard.py

o   Try creating reports for different types of questions

**What You've Accomplished**

Congratulations\! You now have a sophisticated system for extracting and analyzing AI reasoning steps. Your tools can:

·       ✅ **Extract Reasoning Steps**: Identify individual steps in the AI's thinking process

·       ✅ **Classify Step Types**: Categorize steps as understanding, analysis, calculation, etc.

·       ✅ **Assess Quality**: Evaluate the completeness and coherence of reasoning

·       ✅ **Generate Reports**: Create detailed analyses of reasoning quality

·       ✅ **Compare Approaches**: Test how different prompting affects reasoning quality

·       ✅ **Interactive Analysis**: Explore reasoning patterns in real-time

**The Value of Reasoning Extraction**

This system gives you unprecedented insight into how AI thinks through problems. You can:

·       **Debug AI Decisions**: Understand exactly why the AI reached a specific conclusion

·       **Improve Prompts**: Learn what prompting strategies produce better reasoning

·       **Build Trust**: Verify that AI logic is sound before relying on results

·       **Educational Use**: Study effective problem-solving techniques

·       **Quality Assurance**: Ensure AI reasoning meets your standards

Remember, the goal isn't just to get answers—it's to understand the thinking process so you can build more trustworthy, debuggable, and improvable AI systems.

Try experimenting with different types of questions—mathematical problems, logical puzzles, or code analysis tasks. Watch how your agent's reasoning style adapts to different problem types. This hands-on exploration builds intuition about when and how to use reasoning agents effectively in your projects.

In the next chapter, you'll learn how to debug your DeepSeek agent, validate its outputs, and fix common errors that may occur during operation.